# Vérifie que le compilateur nvcc de GPU est bien installé

In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0


# Vérifie qu'on a bien activé le GPU de notre machine (fictive)

In [ ]:
import tensorflow as tf
print("Nombre de GPUs Dispoc: ", len(tf.config.experimental.list_physical_devices('GPU')))

Nombre de GPUs Dispoc:  1


# Installer CUDA


In [ ]:
!apt-get install -y libcudnn8

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  libcudnn8
0 upgraded, 1 newly installed, 0 to remove and 21 not upgraded.
Need to get 444 MB of archives.
After this operation, 1,099 MB of additional disk space will be used.
Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  libcudnn8 8.9.7.29-1+cuda12.2 [444 MB]
Fetched 444 MB in 6s (78.2 MB/s)
Selecting previously unselected package libcudnn8.
(Reading database ... 124926 files and directories currently installed.)
Preparing to unpack .../libcudnn8_8.9.7.29-1+cuda12.2_amd64.deb ...
Unpacking libcudnn8 (8.9.7.29-1+cuda12.2) ...
Setting up libcudnn8 (8.9.7.29-1+cuda12.2) ...


# Nous allons nous servir de ce code pour apprécier cuBLAS


In [ ]:
%%writefile large_matrix_mul.cu
#include <stdio.h>
#include <stdlib.h>
#include <cuda_runtime.h>
#include <cublas_v2.h>
#include <time.h>

#define N 1024  // Taille des matrices

// Fonction pour initialiser la matrice avec des valeurs aléatoires
void initialize_matrix(float *matrix, int size) {
    for (int i = 0; i < size; i++) {
        matrix[i] = (float)(rand() % 100);  // Valeurs entre 0 et 99
    }
}

int main() {
    float *A, *B, *C;
    float *d_A, *d_B, *d_C;

    // Allocation de mémoire pour les matrices sur le CPU
    A = (float*)malloc(N * N * sizeof(float));
    B = (float*)malloc(N * N * sizeof(float));
    C = (float*)malloc(N * N * sizeof(float));

    // Initialisation des matrices A et B avec des valeurs aléatoires
    srand(time(NULL));  // Initialisation de la graine pour rand()
    initialize_matrix(A, N * N);
    initialize_matrix(B, N * N);

    // Création du handle cuBLAS
    cublasHandle_t handle;
    cublasCreate(&handle);

    // Allocation de mémoire sur le GPU
    cudaMalloc((void**)&d_A, sizeof(float) * N * N);
    cudaMalloc((void**)&d_B, sizeof(float) * N * N);
    cudaMalloc((void**)&d_C, sizeof(float) * N * N);

    // Copie des matrices A et B vers le GPU
    cudaMemcpy(d_A, A, sizeof(float) * N * N, cudaMemcpyHostToDevice);
    cudaMemcpy(d_B, B, sizeof(float) * N * N, cudaMemcpyHostToDevice);

    const float alpha = 1.0f, beta = 0.0f;

    // Multiplication de matrices : C = alpha * A * B + beta * C
    cublasSgemm(handle, CUBLAS_OP_N, CUBLAS_OP_N,
                N, N, N,
                &alpha, d_A, N, d_B, N,
                &beta, d_C, N);

    // Copie du résultat de C vers le CPU
    cudaMemcpy(C, d_C, sizeof(float) * N * N, cudaMemcpyDeviceToHost);

    // Affichage partiel (pour ne pas saturer la sortie)
    printf("Quelques éléments de la matrice résultante C:\n");
    for (int i = 0; i < 5; i++) {
        for (int j = 0; j < 5; j++) {
            printf("%8.2f ", C[i * N + j]);
        }
        printf("\n");
    }

    // Libération de la mémoire GPU et CPU
    cudaFree(d_A);
    cudaFree(d_B);
    cudaFree(d_C);
    free(A);
    free(B);
    free(C);
    cublasDestroy(handle);

    return 0;
}


Writing large_matrix_mul.cu


In [ ]:
!nvcc -o large_matrix_mul large_matrix_mul.cu -lcublas

In [ ]:
!./large_matrix_mul

Quelques éléments de la matrice résultante C:
2487438.00 2368418.00 2519780.00 2428181.00 2591257.00 
2543611.00 2506439.00 2522125.00 2509590.00 2713108.00 
2528800.00 2443670.00 2537227.00 2477653.00 2624294.00 
2523920.00 2420775.00 2488090.00 2418354.00 2610738.00 
2456321.00 2315729.00 2422694.00 2397392.00 2530055.00 



N'hésitez pas à jeter un œil à la doc de cuBLAS ! Un vrai IT geek sait toujours où chercher les bonnes infos.

https://docs.nvidia.com/cuda/pdf/CUBLAS_Library.pdf


## 1. Analyse de la structure du code
- **a)** Décrivez les principales étapes du programme (allocation mémoire, transfert de données, calcul, etc.). Pourquoi est-il nécessaire de gérer explicitement la mémoire entre le CPU et le GPU ?
- **b)** Quel est le rôle du `cublasHandle_t` dans ce programme ? Pourquoi doit-on appeler `cublasCreate` et `cublasDestroy` ?
- **c)** Expliquez la signification des paramètres `CUBLAS_OP_N` dans l’appel à `cublasSgemm`. Que se passerait-il si l’un d’eux était remplacé par `CUBLAS_OP_T` (transposition) ?
- **d)** Quel est l’impact de l’ordre de stockage des matrices en mémoire (row-major vs column-major) sur l'utilisation de cuBLAS ?

## 2. Comparaison cuBLAS vs calcul natif
- **a)** Implémentez une version "native" de la multiplication de matrices $ C = A \cdot B $ en C sur le CPU (boucles classiques). Comparez qualitativement la complexité algorithmique de cette approche avec celle de `cublasSgemm`.
- **b)** Modifiez le programme pour mesurer le temps d’exécution de la multiplication avec `cublasSgemm` (utilisez `cudaEvent_t`). Implémentez ensuite votre version native sur CPU et comparez les performances pour $ N = 1024 $. Quelles différences observez-vous, et pourquoi ?
- **c)** Si vous deviez écrire une version parallèle "native" sur GPU (sans cuBLAS, avec un kernel CUDA), quelles seraient les principales difficultés à surmonter par rapport à l’utilisation de cuBLAS ?
- **d)** Est-il pertinent d’utiliser cuBLAS pour des matrices très petites (ex. $ N = 16 $) ? Justifiez votre réponse en considérant les surcharges (overheads) liées au GPU.

## 3. Gestion de la mémoire
- **a)** Pourquoi utilise-t-on `cudaMalloc` et `cudaMemcpy` au lieu de travailler directement avec les matrices $ A $, $ B $, et $ C $ sur le GPU ? Que se passerait-il si on omettait un appel à `cudaMemcpy` avant `cublasSgemm` ?
- **b)** Calculez la quantité totale de mémoire GPU allouée dans ce programme pour $ N = 1024 $ (en Mo). Comment cette consommation évoluerait-elle si $ N $ doublait ?
- **c)** Que risquerait-on en oubliant d’appeler `cudaFree` pour $ d_A $, $ d_B $, et $ d_C $ ? Comment vérifier si la mémoire GPU est bien libérée ?
- **d)** Quelle est la différence entre `cudaMallocManaged` et `cudaMalloc` ? Dans quel cas pourrait-on utiliser `cudaMallocManaged` pour simplifier le transfert de données entre CPU et GPU ?

## 4. Paramètres de cuBLAS
- **a)** Expliquez le rôle des paramètres `alpha` et `beta` dans `cublasSgemm`. Que se passerait-il si `beta = 1.0f` au lieu de `0.0f`, sans initialiser $ d_C $ au préalable ?
- **b)** Modifiez le code pour calculer $ C = 2A \cdot B + 3C $ au lieu de $ C = A \cdot B $. Quels paramètres de `cublasSgemm` faudrait-il ajuster ?
- **c)** Comment adapteriez-vous le code pour multiplier des matrices non carrées, par exemple $ A $ de taille $512 \times 1024$ et $ B $ de taille $ 1024 \times 768 $ ?
- **d)** Existe-t-il d’autres fonctions dans cuBLAS pour la multiplication de matrices en mode batched (plusieurs multiplications en parallèle) ? Comment les utiliser ?

## 5. Optimisation et performance
- **a)** Pourquoi cuBLAS est-il généralement plus rapide qu’une implémentation CPU native pour de grandes matrices comme $N = 1024$ ? Quels aspects du GPU exploite-t-il ?
- **b)** Si $ N $ était beaucoup plus petit (ex. $ N = 16 $), pensez-vous que cuBLAS resterait avantageux par rapport à une version CPU ? Justifiez votre réponse en considérant les overheads (surcharges) liés au GPU.
- **c)** Proposez une modification du code pour traiter des matrices trop grandes pour tenir entièrement dans la mémoire GPU (par exemple, $ N = 10000 $). Comment découperiez-vous le problème ?
- **d)** Comment le choix des blocs et des threads impacte-t-il la performance si on implémente `Sgemm` en kernel CUDA personnalisé ?

## 6. Extensions et réflexion
- **a)** Adaptez le code pour utiliser des nombres en double précision (`double`) avec `cublasDgemm`. Quels changements sont nécessaires dans les types de données et les appels de fonctions ?
- **b)** Imaginez que vous devez multiplier plusieurs paires de matrices consécutivement (ex. $ C = A \cdot B $, puis $ D = C \cdot E $). Comment optimiseriez-vous le code pour minimiser les transferts de données entre CPU et GPU ?
- **c)** Si vous aviez accès à plusieurs GPU, comment modifieriez-vous le code pour distribuer le calcul avec cuBLAS ? Quels défis cela poserait-il ?
- **d)** cuBLAS offre-t-il des optimisations pour les architectures GPU récentes comme Ampere ou Hopper ? Quelles fonctionnalités avancées pourrait-on exploiter ?


## 1. Analyse de la structure du code

**a) Étapes principales et gestion de la mémoire**

- **Allocation et initialisation** : Le programme alloue la mémoire sur le CPU pour les matrices A, B et C, et initialise A et B avec des valeurs aléatoires.
- **Création du contexte cuBLAS** : Un handle (`cublasHandle_t`) est créé avec `cublasCreate` pour gérer l'état de la bibliothèque cuBLAS.
- **Allocation GPU et transfert** : La mémoire est allouée sur le GPU pour les copies de A, B et C à l'aide de `cudaMalloc`. Les données de A et B sont transférées du CPU vers le GPU via `cudaMemcpy`.
- **Calcul sur GPU** : La multiplication matricielle est effectuée sur le GPU en appelant `cublasSgemm`.
- **Retour des résultats** : La matrice résultat C est copiée du GPU vers le CPU.
- **Libération des ressources** : Les mémoires allouées sur le GPU et le CPU sont libérées, et le handle cuBLAS est détruit avec `cublasDestroy`.

La gestion explicite de la mémoire entre le CPU et le GPU est nécessaire car ces deux entités disposent de mémoires physiques séparées. Les données doivent être transférées explicitement pour être accessibles par le GPU.

**b) Rôle du `cublasHandle_t`**

Le `cublasHandle_t` est un objet qui représente le contexte d'exécution de la bibliothèque cuBLAS. Il permet de conserver l'état et les configurations nécessaires pour exécuter les fonctions de la bibliothèque. `cublasCreate` initialise ce contexte et alloue les ressources nécessaires, tandis que `cublasDestroy` libère ces ressources une fois les opérations terminées.

**c) Signification des paramètres `CUBLAS_OP_N`**

Les paramètres `CUBLAS_OP_N` indiquent que les matrices ne doivent pas être transposées lors de l'appel à `cublasSgemm`. Si l’un d’eux était remplacé par `CUBLAS_OP_T`, la matrice correspondante serait transposée avant la multiplication, ce qui modifierait l'ordre des éléments et pourrait changer les dimensions ou les valeurs du résultat final.

**d) Impact de l’ordre de stockage (row-major vs column-major)**

CuBLAS utilise par défaut le format column-major (comme en Fortran), alors que le C standard emploie souvent le format row-major. Ce décalage peut entraîner des interprétations erronées des matrices lors du calcul. Il faut alors soit adapter les paramètres de la fonction (par exemple, en transposant les matrices logiquement), soit convertir explicitement les données.

## 2. Comparaison cuBLAS vs calcul natif

**a) Implémentation native vs cuBLAS**

Une multiplication matricielle native en C impliquerait trois boucles imbriquées, ayant une complexité de O(N³). Bien que la complexité asymptotique soit identique à celle de `cublasSgemm`, ce dernier est hautement optimisé et exploite le parallélisme massif du GPU, ce qui permet d'obtenir des performances bien supérieures pour de grandes matrices.

**b) Mesure du temps d’exécution et comparaison des performances**

En utilisant `cudaEvent_t` pour mesurer le temps d'exécution, on constate généralement que l'exécution avec `cublasSgemm` est beaucoup plus rapide pour N = 1024 par rapport à une implémentation CPU native. Le GPU, grâce à son architecture parallèle, permet de traiter simultanément un grand nombre d'opérations, contrairement au CPU dont les ressources parallèles sont limitées.

**c) Difficultés d'une version parallèle native sur GPU**

Écrire une version native en CUDA impliquerait de gérer manuellement :
- L'organisation des threads et des blocs pour assurer une utilisation efficace des cœurs GPU.
- La gestion de la mémoire partagée pour optimiser les accès aux données.
- La synchronisation entre threads et la minimisation des accès non coalescés.

Ces aspects sont abstraits par cuBLAS, qui offre une implémentation hautement optimisée sans que l'utilisateur ait à gérer ces détails complexes.

**d) Pertinence de cuBLAS pour de petites matrices**

Pour des matrices de petite taille (ex. N = 16), les surcharges liées au lancement de kernels et aux transferts de données entre le CPU et le GPU peuvent être significatives, au point que l'avantage de l'accélération par GPU est perdu. Dans ce cas, une implémentation native sur CPU peut être plus efficace.

## 3. Gestion de la mémoire

**a) Rôle de `cudaMalloc` et `cudaMemcpy`**

Les fonctions `cudaMalloc` et `cudaMemcpy` sont utilisées pour allouer et transférer explicitement les données dans l'espace mémoire du GPU, qui est distinct de celui du CPU. Si on omettait l'appel à `cudaMemcpy` avant `cublasSgemm`, le GPU ne disposerait pas des données initialisées sur le CPU, ce qui conduirait à des erreurs ou à des résultats incorrects.

**b) Calcul de la mémoire GPU allouée pour N = 1024**

Pour chaque matrice :
- Taille = 1024 x 1024 x 4 bytes (pour un float) ≈ 4 Mo.

Pour 3 matrices (A, B, C) : environ 12 Mo au total.

Si N doublait (N = 2048), la mémoire par matrice serait proportionnelle à N², donc environ 4 fois plus élevée, soit environ 48 Mo pour les trois matrices.

**c) Risques en oubliant `cudaFree`**

Omettre d'appeler `cudaFree` pour libérer `d_A`, `d_B` et `d_C` provoquerait une fuite de mémoire sur le GPU, réduisant ainsi la mémoire disponible pour d'autres calculs. On peut vérifier la libération de la mémoire en utilisant des outils comme `nvidia-smi`, qui affichent l'utilisation de la mémoire GPU.

**d) Différence entre `cudaMallocManaged` et `cudaMalloc`**

`cudaMallocManaged` alloue une mémoire unifiée accessible à la fois par le CPU et le GPU, simplifiant ainsi le transfert de données. En revanche, `cudaMalloc` réserve de la mémoire exclusivement sur le GPU, nécessitant des transferts explicites via `cudaMemcpy`.

## 4. Paramètres de cuBLAS

**a) Rôle des paramètres `alpha` et `beta`**

Dans l'appel à `cublasSgemm`, le calcul réalisé est :

    C = alpha * A * B + beta * C

Les paramètres `alpha` et `beta` permettent de scaler respectivement le produit matriciel et la matrice C déjà existante. Si `beta` était fixé à 1.0f sans initialiser C, les valeurs non définies de C seraient ajoutées au résultat, entraînant des erreurs.

**b) Modification pour calculer C = 2A * B + 3C**

Il suffit de remplacer `alpha` par 2.0f et `beta` par 3.0f dans l'appel à `cublasSgemm`.

**c) Adaptation pour des matrices non carrées**

Pour multiplier, par exemple, une matrice A de taille 512x1024 par une matrice B de taille 1024x768, il faut adapter les dimensions dans l'appel à `cublasSgemm` :
- m = 512 (nombre de lignes de A),
- n = 768 (nombre de colonnes de B),
- k = 1024 (nombre de colonnes de A ou lignes de B).

Les paramètres de stride et les leading dimensions doivent également être ajustés en conséquence.

**d) Fonctions batched dans cuBLAS**

CuBLAS propose des fonctions pour la multiplication de matrices en mode batched, telles que `cublasSgemmBatched` et `cublasSgemmStridedBatched`, permettant d'exécuter plusieurs multiplications en parallèle en passant des tableaux de pointeurs vers les matrices ou en utilisant un stride constant.

## 5. Optimisation et performance

**a) Pourquoi cuBLAS est-il plus rapide pour N = 1024 ?**

CuBLAS exploite le parallélisme massif des GPU, utilisant des milliers de cœurs et des optimisations spécifiques (comme la gestion efficace des accès mémoire et l'utilisation de Tensor Cores sur certaines architectures) pour accélérer les calculs de grande envergure.

**b) Avantage de cuBLAS pour de petites matrices (ex. N = 16)**

Pour des matrices de très petite taille, l'overhead associé aux lancements de kernels et aux transferts de données entre CPU et GPU peut surpasser les bénéfices du calcul parallèle. Ainsi, pour N = 16, une implémentation CPU native pourrait être plus performante en raison de la faible charge de calcul et de l'absence d'overhead de transfert.

**c) Traitement de matrices trop grandes pour la mémoire GPU**

Pour des matrices trop grandes (par exemple, N = 10000), il est pertinent de découper le problème en sous-blocs (technique de tiling). Chaque bloc est multiplié séparément sur le GPU et les résultats sont ensuite assemblés pour former la matrice finale. Cette approche permet de traiter des matrices dont la taille excède la mémoire disponible sur le GPU.

**d) Impact du choix des blocs et des threads dans un kernel CUDA personnalisé**

Le choix de la taille des blocs et du nombre de threads influe directement sur l'efficacité de l'utilisation des ressources GPU, la coalescence des accès mémoire et l'utilisation de la mémoire partagée. Une mauvaise configuration peut entraîner une sous-utilisation des capacités du GPU et une dégradation significative des performances.

## 6. Extensions et réflexion

**a) Passage en double précision**

Pour utiliser des nombres en double précision, il faut :
- Remplacer le type `float` par `double` dans la déclaration des matrices.
- Utiliser `cublasDgemm` au lieu de `cublasSgemm`.
- Adapter les constantes `alpha` et `beta` (par exemple, 1.0f deviendra 1.0 en double).

**b) Optimisation pour des multiplications consécutives**

Si plusieurs multiplications doivent être réalisées (par exemple, C = A * B suivi de D = C * E), il est avantageux de conserver les matrices intermédiaires dans la mémoire GPU afin de minimiser les transferts entre le CPU et le GPU.

**c) Utilisation de plusieurs GPU**

Pour exploiter plusieurs GPU, il est nécessaire de partitionner les matrices et de distribuer les calculs entre les différents dispositifs. Les principaux défis incluent la synchronisation inter-GPU, la gestion des transferts de données entre GPU, et la répartition équilibrée de la charge de travail.

**d) Optimisations sur les architectures récentes**

CuBLAS intègre des optimisations pour les architectures récentes comme Ampere ou Hopper, notamment via l'utilisation des Tensor Cores et des fonctionnalités avancées offertes par cuBLASLt, qui permettent un contrôle plus fin et des performances accrues pour certaines opérations de multiplication matricielle.